Videocvičení naleznete zde: https://youtu.be/yL-A0N5JDJo

In [ ]:
%cd /content
%rm -R SKOMAM
!git clone https://github.com/Beremi/SKOMAM
%cd /content/SKOMAM/past_SKOMAM/2021/CV2/
%pwd

# Práce s obrázky

### Načítání balíčků
K práci s obrázky budeme používat knihovnu **cv2** s aliasem **cv**. Dále budeme používat knihovnu **numpy** s aliasem **np** pro matematické funkce a práci s poli a knihovnu **matplotlib** s aliasem **plt** pro vykreslování výsledků. 

Následujícím kódem naimportujeme balíčky. 

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import NoNorm

## Obrázek jako matice

Nejprve je třeba načíst obrázek do paměti. To provedeme příkazem **imread** z knihovny **cv2**.

***Pozor:*** Barevné vrstvy se načtou v pořadí **blue**, **green**, **red** namísto obvyklého **red**, **green**, **blue**. A protože jsme konzervativní, obrázek si převedeme.

In [ ]:
img_bgr = cv.imread("lena_original.jpg",cv.IMREAD_UNCHANGED)
img = cv.cvtColor(img_bgr, cv.COLOR_BGR2RGB)

Chceme-li si obrázek zobrazit, použijeme příkaz **imshow** z knihovny **matplotlib**.

In [ ]:
plt.figure()
plt.imshow(img)

Pro zkoušení algoritmů pro zpracování obrazu se běžně používá jen výřez tohoto obrázku. Ten si teď vyrobíme.

S obrázkem teď můžeme zacházet jako s polem. Můžeme tedy zadat, jaký rozsah indexů chceme nadále používat:

In [ ]:
img_crop = img[20:270,150:400,:]

Výřez si můžeme zase prohlédnout a, protože jej budeme později používat, také uložit.

In [ ]:
plt.figure()
plt.imshow(img_crop)
cv.imwrite('lena_crop.jpg',cv.cvtColor(img_crop, cv.COLOR_RGB2BGR)) 

### Barevné složky

Chceme-li pracovat s některou z barevných složek obrázku, můžeme ji z obrázku získat např. tak, že vynulujeme ostatní složky.

In [ ]:
b = img_crop.copy()
g = img_crop.copy()
r = img_crop.copy()

r[:,:,1] = 0
r[:,:,2] = 0

g[:,:,0] = 0
g[:,:,2] = 0

b[:,:,0] = 0
b[:,:,1] = 0

Výsledné obrázky pak budou vypadat následnovně:

In [ ]:
plt.figure()
plt.imshow(r)

In [ ]:
plt.figure()
plt.imshow(g)

In [ ]:
plt.figure()
plt.imshow(b)

### Základní úpravy obrázku

V následující části cvičení si pro jednoduchost vystačíme s černobílým obrázkem. Načteme si výřez, který jsme si před chvílí uložili. Funkci **imread** ale řekneme, že chceme načíst obrázek jen v odstínech šedé pomocí **cv.IMREAD_GRAYSCALE**.

In [ ]:
img_grey = cv.imread("lena_crop.jpg",cv.IMREAD_GRAYSCALE)

Můžeme si jej vykreslit, ať víme, že vše proběhlo v pořádku. U černobílých obrázků budeme specifikovat, že chceme vykreslit v odstínech šedé pomocí **cmap='gray'** a také, že nechceme, aby během vykreslování došlo k automatickému vyrovnání histogramu (tím se budeme zabývat za chvilku). To se udělá pomocí **norm=NoNorm()**.

In [ ]:
plt.figure()
plt.imshow(img_grey,cmap='gray',norm=NoNorm())

<hr style="border:1px solid black"> </hr>

## Úkol 1: 
vytvořte funkci, která zadaný obrázek zesvětlí o zadaný počet odstínů. Zesvětlený obrázek vykreslete. 

<hr style="border:1px solid black"> </hr>

In [ ]:
def lighten(img, amount):
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if(img[i,j]+amount>254):
                img[i,j]=255
            else:
                img[i,j] += amount 
    return img

In [ ]:
plt.figure()
plt.imshow(lighten(img_grey.copy(),100),cmap='gray',norm=NoNorm())

<hr style="border:1px solid black"> </hr>

## Úkol 2: 
vytvořte funkci, která zadaný obrázek ztmaví o zadaný počet odstínů. Tmavší obrázek vykreslete. 

<hr style="border:1px solid black"> </hr>

In [ ]:
def darken(img, amount):
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if(img[i,j]-amount<1):
                img[i,j] = 0
            else:
                img[i,j] -= amount 
    return img

In [ ]:
plt.figure()
plt.imshow(darken(img_grey.copy(),100),cmap='gray',norm=NoNorm())

<hr style="border:1px solid black"> </hr>

## Úkol 3: 
vytvořte funkci, která vytvoří inverzi (negativ) zadaného obrázku. Negativ vykreslete. 

<hr style="border:1px solid black"> </hr>

In [ ]:
def invert(img):
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            img[i,j] = 255-img[i,j]
    return img

In [ ]:
plt.figure()
plt.imshow(invert(img_grey.copy()),cmap='gray',norm=NoNorm())

### Prahování

Prahování spočívá v tom, že se všechny hodnoty obrázku, které jsou menší než námi zvolený práh (threshold), nastaví na černou, zatímco se zbývanící hodnoty nastaví na bílou.

<hr style="border:1px solid black"> </hr>

## Úkol 4: 
vytvořte funkci, která provede prahování obrázku pomocí zadané hodnoty. Výsledný obrázek vykreslete. 

<hr style="border:1px solid black"> </hr>

In [ ]:
def threshold(img, threshold):
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if(img[i,j]<threshold):
                img[i,j]=0
            else:
                img[i,j]=255
    return img

In [ ]:
plt.figure()
plt.imshow(threshold(img_grey.copy(),100),cmap='gray',norm=NoNorm())

## Vyrovnání histogramu

### Získání histogramu

Nejprve si obrázek načteme. Nezapomeňte, že jej chceme v odstínech šedé.

In [ ]:
img_uneq = cv.imread("uneq.jpg", cv.IMREAD_GRAYSCALE)

Provedeme kontrolu vykreslením.

In [ ]:
plt.figure()
plt.imshow(img_uneq,cmap='gray',norm=NoNorm())

Vidíme, že s obrázkem něco není v pořádku. Zkusíme se podívat na jeho histogram. Histogram obrázku získáme tak, že pro každou možnou hodnotu jasu (v obrázku jich je dohromady 256, 0 reprezentuje černou, 255 bílou) spočítáme, kolikrát se v obrázku vyskytuje.

<hr style="border:1px solid black"> </hr>

## Úkol 5: 
vytvořte funkci, která k zadanému obrázku vytvoří histogram a použijte ji na obrázek **img_uneq**. Výsledný histogram vykreslete. 

***Nápověda***: k vykreslení histogramu se hodí použít sloupcový graf. Ten získáme pomocí příkazu **bar** z knihovny **matplotlib**.

<hr style="border:1px solid black"> </hr>

In [ ]:
def get_hist(image):
    hist = np.zeros(256)
    for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                hist[int(image[i,j])] += 1
    return hist

hist_uneq = get_hist(img_uneq)

plt.figure()
plt.bar([i for i in range(hist_uneq.shape[0])],hist_uneq/float(sum(hist_uneq)))

### Vyrovnání histogramu

Z obrázku vidíme, že v obrázku je jen úzký rozsah jasu. Pokusíme se toto napravit a histogram tzv. vyrovnáme. Cílem je celý histogram roztáhnout tak, aby pokrýval celý rozsah of 0 do 255.

<hr style="border:1px solid black"> </hr>

## Úkol 6: 
vytvořte funkci, která vyrovná histogram zadaného obrázku a použijte ji na obrázek **img_uneq**. Výsledný obrázek a histogram vykreslete. 

<hr style="border:1px solid black"> </hr>

In [ ]:
def eq_hist(img):    
    hist = get_hist(img)
    i=0
    while(hist[i]==0):
        i+=1
    j=255
    while(hist[j]==0):
        j-=1
    for k in range(img.shape[0]):
            for l in range(img.shape[1]):
                img[k,l] = (img[k,l]-i)*(255.0/(j-i))                
    return img

img_eq = eq_hist(img_uneq.copy())
plt.figure()
plt.imshow(img_eq,cmap='gray',norm=NoNorm())

hist_eq = get_hist(img_eq)
plt.figure()
plt.bar([i for i in range(hist_eq.shape[0])],hist_eq/float(sum(hist_eq)))

## Konvoluce

Diskrétní konvoluce je operace, která obrázek modifikuje pomocí takzvané konvoluční masky. Konvoluční masku si můžeme představit jako čtvercovou matici, jejíž hodnoty představují váhy, s jakými do výsledného obrázku započítáváme hodnoty jasu obrázku původního.

V praxi konvoluce funguje tak, že masku přiložíme na obrázek tak, aby její střed byl v bodě, pro který chceme konvoluci počítat. Hodotu pak získáme tím, že po složkách vynásobíme masku s jasy obrázku a vše sečteme. Obrázek se schématem najdete na Wikipedii, ze které jsem si jej vypůjčil.

![Schéma diskrétní konvoluce (zdroj: Wikipedie)](https://github.com/Beremi/SKOMAM/blob/main/past_SKOMAM/2021/CV2/konvoluce.jpg?raw=1)

Naším úkolem nebude nic jiného, než diskrétní konvoluci naprogramovat a otestovat s různými maskami. Ty si ostatně můžeme rovnou zadefinovat.

In [ ]:
average = np.array([[1, 1, 1],[1, 1, 1],[1, 1, 1]])
gauss_large = np.array([[1, 4, 7, 4, 1],[4, 16, 26, 16, 4],[7, 26, 41, 26, 7],[4, 16, 26, 16, 4],[1, 4, 7, 4, 1]])
gauss = np.array([[1, 2, 1],[2, 4, 2],[1, 2, 1]])
laplace = np.array([[0, 1, 0],[1, -4, 1],[0, 1, 0]])
edges = np.array([[0, -1, 0],[-1, 5, -1],[0, -1, 0]])
vertical_edges = np.array([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]])
horizontal_edges = np.array([[-1, -2, -1],[0, 0, 0],[1, 2, 1]])

Různé masky se hodí na různé operace, proto si je vyzkoušíme na dvou různých obrázcích. Jeden z nich bude zašuměná Lena

In [ ]:
lena = cv.imread("lena_noise.jpg", cv.IMREAD_GRAYSCALE)

Druhý obrázek bude obrázek cihlové zdi, na kterém bude dobře vidět efekt masek, které zvýrazňují hrany.

In [ ]:
bricks = cv.imread("bricks.jpg", cv.IMREAD_GRAYSCALE)

<hr style="border:1px solid black"> </hr>

## Úkol 7: 
vytvořte funkci, která provede diskrétní konvoluci zadaného obrázku se zadanou konvoluční maskou. Otestujte efekty různých konvolučních masek.

<hr style="border:1px solid black"> </hr>

In [ ]:
def convolution(img, mask):
    
    conv = np.zeros(img.shape)
    l = int(mask.shape[0]/2)
    
    for i in range(l ,img.shape[0]-l):
        for j in range(l,img.shape[1]-l):
            conv[i,j] = int(sum(sum(img[(i-l):(i+l+1),(j-l):(j+l+1)]*mask)))

    return conv

In [ ]:
img = bricks
mask = edges

In [ ]:
img_conv = convolution(img.copy(),mask)

In [ ]:
plt.figure()
plt.imshow(img,cmap='gray')

In [ ]:
plt.figure()
plt.imshow(img_conv,cmap='gray')